In [64]:
import urllib.request
import requests
from IPython.display import display, Math, Latex
from tqdm import tqdm
import seaborn as sns
import geopandas as gpd
import math
import time as ti
import json
import os
import sys

import branca.colormap as cm
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point, Polygon

sys.path.insert(0, "../src")


pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)


# Now let's try to compute the distance OD Matrix between these points

First, let's define the function need to perform the HTTP GET request. All the assumptions regarding this matrix are under this first link below.

- Resources: https://docs.microsoft.com/en-us/bingmaps/rest-services/examples/distance-matrix-asynchronous-example

- Example: https://docs.microsoft.com/en-us/bingmaps/rest-services/examples/distance-matrix-example


The HTTP GET request works as follow:

https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?origins={lat0,long0;lat1,lon1;latM,lonM}&destinations={lat0,lon0;lat1,lon1;latN,longN}&travelMode={travelMode}&startTime={startTime}&timeUnit={timeUnit}&key={BingMapsKey}

- origins: list of lat, long coordinates of origins
- destionations: list of lat, long coordinates of destinations
- travelMode: `Driving`
- startTime: `2021-11-15-07:00`
- timeUnit
- BingMapsKey


In [65]:
def get_data(orig, dest, size_x, size_y):
    # Your Bing Maps Key
    bingMapsKey = "Ah52Budr_8V_sE1HKdPH-INIK_RDqas1zgkSJwdtB3GW3Dm04DsNm3lzkoXmpPc9"

    # input information
    travelMode = "Driving"
    startTime = "2021-11-15-07:00"
    distanceUnit = "km"
    timeUnit = "minute"

    routeUrl = (
        "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?"
        + f"origins={orig}"
        + f"&destinations={dest}"
        + f"&travelMode={travelMode}"
        + f"&startTime={startTime}"
        + f"&timeUnit={timeUnit}"
        + f"&key={bingMapsKey}"
    )

    # request = urllib.request.Request(routeUrl)
    # response = urllib.request.urlopen(request, proxies={"http":"http://myproxy:3129"})
    # data = json.load(response)

    response = requests.get(routeUrl)
    data = response.json()

    distance, time = build_od_matrix(data, (size_x, size_y))
    return distance, time

In [66]:
def build_od_matrix(data, size):

    resources = data["resourceSets"][0]["resources"][0]["results"]

    distance = np.zeros((size[0], size[1]))
    time = np.zeros((size[0], size[1]))

    for i in range(size[0]):
        for j in range(size[1]):
            distance[i, j] = resources[i * size[1] + j]["travelDistance"]
            time[i, j] = resources[i * size[1] + j]["travelDuration"]

    return distance, time

# Loading the data


In [67]:
# load the housing data
residences = pd.read_csv("../data/HousingData.csv")
residences = residences[residences["Residence"] != 'Simmons Hall']
residences.loc[len(residences.index)] = ["Depot", "139 Massachusetts Ave, Cambridge, MA 02138", 42.36427095750911, -71.10169629169343, "NA", "NA"]

mit_buildings = pd.read_csv("../data/MIT_Buildings.csv")
shuttle_stops = pd.read_csv("../data/current_stops.csv").rename(
    columns={"Station Name": "Name"}
)

origins = residences[["Residence", "Long", "Lat"]].rename(
    columns={"Residence": "Name"})

# add a row to origins
destinations = shuttle_stops.append(
    residences[["Residence", "Lat", "Long"]].rename(
        columns={"Residence": "Name"})
)
destinations = destinations.append(mit_buildings[["Name", "Lat", "Long"]])
# destinations.loc[len(destinations.index)] = ["Depot", 42.36381606868144, -71.0885840857672]

# save origins and destionations to CSV
origins.to_csv("../data/origins.csv", index=False)
destinations.to_csv("../data/destinations.csv", index=False)

/var/folders/hk/nxk7bnsj4kq9nwsmnmdtzhb80000gn/T/ipykernel_11859/298089451.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destinations = shuttle_stops.append(
/var/folders/hk/nxk7bnsj4kq9nwsmnmdtzhb80000gn/T/ipykernel_11859/298089451.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  destinations = destinations.append(mit_buildings[["Name", "Lat", "Long"]])


In [68]:
N_o = destinations.shape[0]
N_d = destinations.shape[0]

# Matrix Initialization
distance_matrix = np.zeros((N_o, N_d))
time_matrix = np.zeros((N_o, N_d))


In [69]:

# or_points = []
# for i, j in zip(lat, long):
#     or_points.append(str(i) + "," + str(j))

# # Compute the list of origin-destinations points
# long = list(destinations["Long"])
# lat = list(destinations["Lat"])

# des_points = []
# for i, j in zip(lat, long):
#     des_points.append(str(i) + "," + str(j))

# or_points = ";".join(or_points)
# des_points = ";".join(des_points)

In [70]:
total = 45
N = 25
Nn = math.ceil(total/N)

# Matrix Initialization
distance_matrix = np.zeros((total, total))
time_matrix = np.zeros((total, total))

# Compute the list of origin-destinations points
long = list(destinations["Long"])
lat = list(destinations["Lat"])

points = []
for i, j in zip(lat, long):
    points.append(str(i) + "," + str(j))

## Querying BingMaps Services
for i in tqdm(range(Nn)):  # division in 6 batches

    min_, max_ = int(i * N), int(min(total, (i + 1) * N))
    subset_origins = ";".join(points[min_:max_])

    for j in range(Nn):
        # ti.sleep(5)

        min_j, max_j = int(j * N), int(min(total, (j + 1) * N))
        subset_destinations = ";".join(points[min_j:max_j])

        print(min_, max_, min_j, max_j)

        distance, time = get_data(
            subset_origins, subset_destinations, size_x=int(max_ - min_), size_y=int(max_j - min_j)
        )


        distance_matrix[min_:max_, min_j:max_j] = distance
        time_matrix[min_:max_, min_j:max_j] = time

np.savetxt("../data/BingMapsAPI_Test3.txt", distance_matrix, fmt="%1.3f")

  0%|          | 0/2 [00:00<?, ?it/s]

0 25 0 25
0 25 25 45


 50%|█████     | 1/2 [00:02<00:02,  2.20s/it]

25 45 0 25
25 45 25 45


100%|██████████| 2/2 [00:04<00:00,  2.14s/it]


In [71]:
dists.head()

Name,Kendall Square T,Wadsworth at Amherst,Media Lab,Media Lab at Ames,Amherst at Kresge,Burton/MacGregor,Tang/Westgate,W98 at Vassar St,W92 at Amesbury St,Simmons Hall,New Vassar Dorm,Vassar St at Mass Ave,Main at Vassar,Baker House,Burton Conner,East Campus,MacGregor House,Maseeh Hall,McCormick Hall,New House,Next House,New Vassar,Random Hall,70 Amherst Street,Ashdown House,Edgerton House,Graduate Tower,Sidney-Pacific,Tang Hall,The Warehouse,Westgate Apartments,Depot,MIT Chapel,Hyatt Regency Cambridge,MIT Furniture Exchange,Veterans Memorial Pool,Great Dome,MIT Sloan School of Management,MIT Medical Urgent Care Service,Walker Memorial,MIT Department of Mathematics,MIT Sailing Pavilion,The Muddy Charles Pub,Tudor Dog Park,MIT Plasma Science and Fusion Center
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Kendall Square T,0.000,0.301,0.505,0.752,1.344,1.828,2.617,2.565,2.697,2.075,1.764,1.608,2.047,1.486,1.669,0.633,1.835,1.291,1.513,1.932,2.043,2.288,1.872,0.521,2.115,1.823,0.105,2.226,2.233,2.063,2.552,2.241,1.365,2.360,2.921,3.215,1.650,0.303,0.543,0.647,0.896,2.299,0.783,2.426,1.863
Wadsworth at Amherst,0.727,0.000,0.203,0.288,1.085,1.569,2.358,2.306,2.438,1.816,1.505,1.349,1.788,1.227,1.410,0.331,1.576,1.032,1.254,1.673,1.784,2.029,1.613,0.219,1.856,1.564,0.215,1.967,1.974,1.804,2.293,1.982,1.106,2.101,2.662,2.956,1.391,0.184,0.241,0.345,0.637,2.040,0.319,2.167,1.604
Media Lab,0.523,0.203,0.000,0.084,0.840,1.324,2.113,2.061,2.193,1.571,1.260,0.919,1.016,0.982,1.165,0.127,1.331,0.787,1.009,1.428,1.539,1.726,1.368,0.016,1.553,1.261,0.629,1.664,1.729,1.501,2.048,1.567,0.861,1.856,2.417,2.711,0.506,0.388,0.130,0.141,0.392,1.795,0.115,1.858,1.301
Media Lab at Ames,0.477,0.288,0.084,0.000,0.766,1.250,2.039,1.987,2.119,1.497,1.186,1.030,1.469,0.908,1.091,0.081,1.257,0.713,0.935,1.354,1.465,1.710,1.294,0.068,1.537,1.245,0.583,1.648,1.655,1.485,1.974,1.663,0.787,1.782,2.343,2.637,1.072,0.781,0.215,0.095,0.318,1.721,0.031,1.848,1.285
Amherst at Kresge,1.177,1.152,1.491,1.417,0.000,0.664,1.453,1.401,1.533,0.921,0.610,0.454,0.893,0.192,0.378,1.488,0.671,0.052,0.040,0.768,0.879,1.134,0.718,1.475,0.961,0.669,1.283,1.072,1.069,0.909,1.388,1.087,0.103,1.196,1.757,2.051,0.496,1.294,1.394,1.502,0.932,0.647,1.448,1.272,0.709


In [72]:
dists = pd.DataFrame(distance_matrix, columns=destinations["Name"])
dists.set_index(destinations["Name"], inplace=True)

times = pd.DataFrame(time_matrix, columns=destinations["Name"])
times.set_index(destinations["Name"], inplace=True)

#Save data
dists.to_csv("../data/OR_Full_Distances.csv")
times.to_csv("../data/OR_Full_Times.csv")

In [76]:
dists.head()

Name,Kendall Square T,Wadsworth at Amherst,Media Lab,Media Lab at Ames,Amherst at Kresge,Burton/MacGregor,Tang/Westgate,W98 at Vassar St,W92 at Amesbury St,Simmons Hall,New Vassar Dorm,Vassar St at Mass Ave,Main at Vassar,Baker House,Burton Conner,East Campus,MacGregor House,Maseeh Hall,McCormick Hall,New House,Next House,New Vassar,Random Hall,70 Amherst Street,Ashdown House,Edgerton House,Graduate Tower,Sidney-Pacific,Tang Hall,The Warehouse,Westgate Apartments,Depot,MIT Chapel,Hyatt Regency Cambridge,MIT Furniture Exchange,Veterans Memorial Pool,Great Dome,MIT Sloan School of Management,MIT Medical Urgent Care Service,Walker Memorial,MIT Department of Mathematics,MIT Sailing Pavilion,The Muddy Charles Pub,Tudor Dog Park,MIT Plasma Science and Fusion Center
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Kendall Square T,0.000,0.301,0.505,0.752,1.344,1.828,2.617,2.565,2.697,2.075,1.764,1.608,2.047,1.486,1.669,0.633,1.835,1.291,1.513,1.932,2.043,2.288,1.872,0.521,2.115,1.823,0.105,2.226,2.233,2.063,2.552,2.241,1.365,2.360,2.921,3.215,1.650,0.303,0.543,0.647,0.896,2.299,0.783,2.426,1.863
Wadsworth at Amherst,0.727,0.000,0.203,0.288,1.085,1.569,2.358,2.306,2.438,1.816,1.505,1.349,1.788,1.227,1.410,0.331,1.576,1.032,1.254,1.673,1.784,2.029,1.613,0.219,1.856,1.564,0.215,1.967,1.974,1.804,2.293,1.982,1.106,2.101,2.662,2.956,1.391,0.184,0.241,0.345,0.637,2.040,0.319,2.167,1.604
Media Lab,0.523,0.203,0.000,0.084,0.840,1.324,2.113,2.061,2.193,1.571,1.260,0.919,1.016,0.982,1.165,0.127,1.331,0.787,1.009,1.428,1.539,1.726,1.368,0.016,1.553,1.261,0.629,1.664,1.729,1.501,2.048,1.567,0.861,1.856,2.417,2.711,0.506,0.388,0.130,0.141,0.392,1.795,0.115,1.858,1.301
Media Lab at Ames,0.477,0.288,0.084,0.000,0.766,1.250,2.039,1.987,2.119,1.497,1.186,1.030,1.469,0.908,1.091,0.081,1.257,0.713,0.935,1.354,1.465,1.710,1.294,0.068,1.537,1.245,0.583,1.648,1.655,1.485,1.974,1.663,0.787,1.782,2.343,2.637,1.072,0.781,0.215,0.095,0.318,1.721,0.031,1.848,1.285
Amherst at Kresge,1.177,1.152,1.491,1.417,0.000,0.664,1.453,1.401,1.533,0.921,0.610,0.454,0.893,0.192,0.378,1.488,0.671,0.052,0.040,0.768,0.879,1.134,0.718,1.475,0.961,0.669,1.283,1.072,1.069,0.909,1.388,1.087,0.103,1.196,1.757,2.051,0.496,1.294,1.394,1.502,0.932,0.647,1.448,1.272,0.709


In [74]:
# distance_matrix, time_matrix = get_data(
#     des_points, des_points, size_x=int(N_d), size_y=int(N_d)
# )

In [75]:
distances = pd.DataFrame(distance_matrix)

# give a destination to each column name
distances.columns = destinations["Name"]
distances.index = origins["Name"]

# Reshape dataframe to long dataframe
distances = distances.reset_index()
distances = pd.melt(
    distances,
    id_vars=["Name"],
    value_vars=distances.columns.values.tolist()[1:],
    var_name="Destination",
    value_name="Distance",
)

ValueError: Length mismatch: Expected axis has 45 elements, new values have 19 elements

In [ ]:
times = pd.DataFrame(time_matrix)

# give a destination to each column name
times.columns = destinations["Name"]
times.index = origins["Name"]

# Reshape dataframe to long dataframe
times = times.reset_index()
times.to_csv("../data/origin_destination_times.csv", index=False)
times = pd.melt(
    times,
    id_vars=["Name"],
    value_vars=times.columns.values.tolist()[1:],
    var_name="Destination",
    value_name="Time (min)",
)

In [ ]:
distances["Time (min)"] = times["Time (min)"]
distances = distances.rename(columns={"Name": "Origin"})
distances.head()


,Origin,Destination,Distance,Time (min)
0,Baker House,Kendall Square T,2.752,7.6167
1,Burton Conner,Kendall Square T,2.578,8.0167
2,East Campus,Kendall Square T,0.520,1.8500
3,MacGregor House,Kendall Square T,2.497,7.9667
4,Maseeh Hall,Kendall Square T,1.124,3.9333


In [ ]:
distances.to_csv("../data/origin_destination_distances.csv", index=False)

In [ ]:
# show more rows
pd.set_option("display.max_rows", 500)

times

,Name,Destination,Time (min)
0,Baker House,Kendall Square T,7.6167
1,Burton Conner,Kendall Square T,8.0167
2,East Campus,Kendall Square T,1.8500
3,MacGregor House,Kendall Square T,7.9667
4,Maseeh Hall,Kendall Square T,3.9333
...,...,...,...
850,Sidney-Pacific,MIT Plasma Science and Fusion Center,1.7333
851,Tang Hall,MIT Plasma Science and Fusion Center,4.8167
852,The Warehouse,MIT Plasma Science and Fusion Center,1.3000
853,Westgate Apartments,MIT Plasma Science and Fusion Center,4.1167


In [ ]:
times = pd.DataFrame(time_matrix)
times.columns = destinations["Name"]
times.index = origins["Name"]

# Reshape dataframe to long dataframe
times = times.reset_index()


In [ ]:
times

Name,Name,Kendall Square T,Wadsworth at Amherst,Media Lab,Media Lab at Ames,Amherst at Kresge,Burton/MacGregor,Tang/Westgate,W98 at Vassar St,W92 at Amesbury St,Simmons Hall,New Vassar Dorm,Vassar St at Mass Ave,Main at Vassar,Baker House,Burton Conner,East Campus,MacGregor House,Maseeh Hall,McCormick Hall,New House,Next House,New Vassar,Random Hall,70 Amherst Street,Ashdown House,Edgerton House,Graduate Tower,Sidney-Pacific,Tang Hall,The Warehouse,Westgate Apartments,Depot,MIT Chapel,Hyatt Regency Cambridge,MIT Furniture Exchange,Veterans Memorial Pool,Great Dome,MIT Sloan School of Management,MIT Medical Urgent Care Service,Walker Memorial,MIT Department of Mathematics,MIT Sailing Pavilion,The Muddy Charles Pub,Tudor Dog Park,MIT Plasma Science and Fusion Center
0,Baker House,7.6167,7.7833,8.5167,8.3500,6.8000,3.2833,4.2667,3.3333,3.4833,3.9000,4.5833,5.7500,6.9000,0.0000,1.5167,8.6167,3.2833,6.5833,7.0667,3.2833,4.2667,8.1500,7.0667,8.5167,6.1333,6.8667,7.9833,6.7500,3.0333,6.3833,3.8667,7.8167,7.3833,3.3667,4.1833,5.0333,7.6500,8.4167,10.1500,8.6167,6.9167,6.7833,8.3500,6.1833,7.1833
1,Burton Conner,8.0167,8.1833,8.9167,8.7667,7.2000,1.7667,2.7333,3.7500,3.8833,4.3000,4.9833,6.1667,7.3000,9.2333,0.0000,9.0167,1.7667,7.0000,7.4833,1.7667,2.7333,8.5667,7.4667,8.9167,6.5333,7.2667,8.3833,7.1500,3.4333,6.8000,4.2833,8.2333,7.7833,3.7667,4.5833,5.4333,8.0667,8.8167,10.5667,9.0167,7.3167,7.2000,8.7667,6.5833,7.5833
2,East Campus,1.8500,1.8500,0.6000,0.7667,2.9833,3.2833,5.1667,4.1167,4.2500,4.7333,4.3667,3.7833,4.9000,3.4333,3.0667,0.0000,3.2833,2.7833,2.9167,3.2833,4.2500,7.5500,4.9000,0.6000,5.6333,4.8000,2.2167,6.1667,3.8167,5.7833,4.6500,6.3833,3.5667,4.1333,4.9833,5.8333,4.7500,3.3333,2.1500,0.2333,1.2333,4.4833,0.7667,7.2500,5.0167
3,MacGregor House,7.9667,8.1333,8.8667,8.7000,7.1500,0.1167,0.9667,3.7000,3.8333,4.2500,4.9333,6.1167,7.2500,9.1667,8.7833,8.9667,0.0000,6.9500,7.4333,1.6000,0.9667,8.5000,7.4167,8.8667,6.4833,7.2167,8.3333,7.1000,3.3833,6.7500,0.8167,8.1667,7.7333,3.7167,4.5333,5.3833,8.0000,8.7667,10.5000,8.9667,7.2667,7.1500,8.7000,6.5333,7.5333
4,Maseeh Hall,3.9333,3.2333,3.9667,4.2000,0.2667,2.6500,4.5500,3.5000,3.6333,3.0000,2.6333,2.0500,3.2333,2.2333,2.2667,4.6000,2.6500,0.0000,0.5333,2.6500,3.6333,5.8167,3.1667,3.9667,3.9000,3.0667,4.3167,4.4333,3.1833,4.0500,4.0333,4.6500,1.0167,3.5167,4.3500,5.2167,3.0167,3.8667,5.6167,4.6000,2.4167,2.3000,4.2000,5.5167,3.2833
5,McCormick Hall,4.5333,3.8333,4.5667,4.8000,0.4167,2.9833,4.1333,3.0833,3.2167,3.6000,3.2333,2.6500,3.8333,1.6833,1.7167,5.1833,2.9833,0.6500,0.0000,2.9833,3.9667,6.4167,3.7667,4.5667,4.5000,3.6667,4.9167,5.0333,2.7833,4.6500,3.6167,5.2500,1.7833,3.1000,3.9167,4.7667,3.6167,4.4667,6.2167,5.1833,3.0167,2.9000,4.8000,6.1167,3.8833
6,New House,5.3667,6.4333,6.0500,6.1167,4.5500,1.7333,0.9667,3.7000,2.0833,1.8167,2.3333,3.5167,4.6667,6.5833,6.1833,6.3667,1.6000,4.3500,4.8333,0.0000,0.9667,8.5000,4.8333,6.0500,6.4833,4.7333,5.7500,7.1000,3.3833,6.7500,0.8167,6.3167,5.1500,3.7167,4.5333,5.3833,5.4167,6.8500,6.8167,6.3667,7.2667,7.1500,6.1167,6.5333,4.9500
7,Next House,4.8833,5.9500,5.5667,5.6333,4.0667,1.9000,0.8000,1.8667,1.6000,1.3333,1.8500,3.0333,4.1833,6.1000,5.7000,5.8833,1.9000,3.8667,4.3500,1.9000,0.0000,7.0000,4.3500,5.5667,5.0833,4.2500,5.2667,5.6167,2.1000,5.2333,0.3333,5.8333,4.6667,2.3667,3.8167,4.6667,4.9333,6.3667,6.3333,5.8833,5.3333,5.2167,5.6333,5.8000,4.4667
8,New Vassar,4.8667,5.9167,5.5167,5.5833,4.2833,6.1167,5.6500,5.3333,5.0833,3.9667,3.6000,3.5167,4.0833,6.3167,5.9000,5.8500,6.1167,4.0833,4.5667,6.1167,5.6500,0.0000,3.4000,5.5167,1.5000,1.5667,5.2333,1.6833,5.5833,0.5500,5.1333,4.4167,4.8833,5.8500,5.1833,7.4167,5.1500,6.3500,6.3167,5.8500,5.8833,5.7667,5.5833,2.7667,1.8667
9,Random Hall,3.8833,4.9333,4.5333,4.6000,3.0333,4.8667,4.3833,4.0833,3.8333,2.7167,2.3500,2.2667,3.0833,5.0500,4.6500,4.8500,4.8667,2.8333,3.3167,4.8667,4.3833,4.3000,0.0000,4.5333,2.3833,1.5500,4.2500,1.9833,4.3167,2.5333,3.8667,1.9500,3.6167,4.5833,5.31